# Data Cleaning - Streamly Case Study

## Overview
This notebook documents the data cleaning process for the Streamly recommendation system. I extract three raw CSV files, identify data quality issues, and produce cleaned datasets ready for database ingestion.

## Cleaning Objectives
1. Handle missing values appropriately
2. Convert data types to correct formats
3. Remove duplicate records

In [17]:
import pandas as pd

In [18]:
results = pd.read_csv("../data/results.csv", header=None)

In [19]:
results.columns = [
    "timestamp", "client_id", "email", "invoice_id", "period",
    "product", "entry_type", "environment", "baseline_value",
    "actual_value", "currency"
]

In [20]:
profiles = pd.read_csv("../data/profiles.csv")

## Data Cleaning Steps

### 1. Results Dataset (results.csv)
**Issues Identified:**
- Timestamp fields stored as strings - need datetime conversion
- Period field contains invalid date formats - `pd.to_datetime(..., errors='coerce')` handles conversion failures
- Monetary values (baseline_value, actual_value) stored as strings - converted to numeric with coercion
- Missing product and entry_type values - filled with "Unknown" placeholder to maintain referential integrity
- Inconsistent case and whitespace in categorical fields

**Actions Taken:**
- Convert timestamp and period columns to datetime format
- Convert baseline_value and actual_value to float for calculations
- Fill missing categorical values with "Unknown" to preserve row count
- Maintain client_id and email for account relationships

In [11]:
titles = pd.read_csv("../data/titles.csv")

### 2. Profiles Dataset (profiles.csv)
**Issues Identified:**
- Missing values in age_band, preferred_language, and preferences columns
- Duplicate profile entries across accounts


**Actions Taken:**
- Keep missing values in preference fields (NULL) to allow conditional filtering in recommendations
- Remove exact duplicate rows using `drop_duplicates()`


### 3. Titles Dataset (titles.csv)
**Issues Identified:**
- Missing values in some columns


**Actions Taken:**
- Keep missing regions and languages (NULL) - will be filtered during recommendation queries


In [25]:


try:
    # Profiles: keep preference-related missing values
    if 'profiles' in globals():
        pref_cols = ['age_band', 'preferred_language', 'preferences']
        for c in pref_cols:
            if c in profiles.columns:
                # ensure missing values remain as NaN/None (no imputation)
                profiles[c] = profiles[c].where(profiles[c].notna(), None)

        # Remove exact duplicate profile rows
        prof_before = len(profiles)
        profiles = profiles.drop_duplicates()
        prof_after = len(profiles)
        print(f"Profiles: dropped {prof_before - prof_after} exact duplicate rows ({prof_before} -> {prof_after})")
    else:
        print('Profiles DataFrame not found in notebook namespace; ensure it is loaded earlier.')
except Exception as e:
    print('Error while cleaning profiles:', e)

try:
    # Titles: explicitly preserve missing 'region' and 'language' 
    if 'titles' in globals():
        for c in ['region', 'language']:
            if c in titles.columns:
                titles[c] = titles[c].where(titles[c].notna(), None)
        print('Titles: preserved missing values for region/language where present')
        
        titles_before = len(titles)
        titles = titles.drop_duplicates()
        titles_after = len(titles)
        print(f"Profiles: dropped {titles_before - titles_after} exact duplicate rows ({titles_after} -> {titles_after})")
    else:
        print('Titles DataFrame not found in notebook namespace; ensure it is loaded earlier.')
except Exception as e:
    print('Error while processing titles:', e)

Profiles: dropped 0 exact duplicate rows (359 -> 359)
Titles: preserved missing values for region/language where present
Profiles: dropped 0 exact duplicate rows (1000 -> 1000)


In [23]:
# --- Clean results.csv ---
results["timestamp"] = pd.to_datetime(results["timestamp"], errors="coerce")
results["period"] = pd.to_datetime(results["period"], errors="coerce")

results["baseline_value"] = pd.to_numeric(results["baseline_value"], errors="coerce")
results["actual_value"] = pd.to_numeric(results["actual_value"], errors="coerce")

results["product"] = results["product"].fillna("Unknown")
results["entry_type"] = results["entry_type"].fillna("Unknown")

In [15]:
# Save cleaned datasets
results.to_csv("../data/results_clean.csv", index=False)
profiles.to_csv("../data/profiles_clean.csv", index=False)
titles.to_csv("../data/titles_clean.csv", index=False)

print('\nSaved cleaned files to ../data/')


Saved cleaned files to ../data/
